In [ ]:
import os

from matplotlib import ticker
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from mpl_toolkits.mplot3d.art3d import pathpatch_2d_to_3d
import numpy as np
import pandas as pd
import seaborn as sns

from dosipy.utils.viz import (fig_config, set_axes_equal, set_colorblind,
                              save_fig)

from utils import load_processed_data, load_apd_data

In [ ]:
set_colorblind()
%config InlineBackend.figure_format = 'retina'

In [ ]:
# load data
ds = load_apd_data()
ds

# PD distribution

In [ ]:
# visuals - apd distributions

edge = 0.02
orientation = 'Horizontal'
ds_partial = ds[ds['orientation']==orientation].reset_index()
show_labels = False
for i in range(ds_partial.shape[0]):
    # extract input data
    antenna = ds_partial.loc[i, 'antenna']
    orientation = ds_partial.loc[i, 'orientation']
    distance = ds_partial.loc[i, 'distance']
    origin = eval(ds_partial.loc[i, 'origin'])
    
    # extract point cloud and power density distribution data
    df = load_processed_data(f'{antenna}{orientation}', distance)
    xyz = df[['x', 'y', 'z']].values
    PD = df['PD'].values
    
    # set up figure
    fig_config(latex=True, text_size=24, scaler=2)
    fig = plt.figure()
    ax = plt.axes(projection ='3d')

    # point cloud and power ddensity spatial distribution
    cs = ax.scatter(-xyz[:, 0], -xyz[:, 1], xyz[:, 2],
                    c=PD.ravel(), cmap='mako', s=0.75)

    # set averaging surface contours
    rect_pos = xyz[:, 0].max()
    patch_rect = Rectangle(origin, edge, edge,
                           ec='w', fc='None', lw=3)
    ax.add_patch(patch_rect)
    pathpatch_2d_to_3d(patch_rect, z=rect_pos, zdir='x')

    # set colorbar
    cbar = fig.colorbar(cs, shrink=0.65, pad=-0.05)
    tick_locator = ticker.MaxNLocator(nbins=4)
    cbar.locator = tick_locator
    cbar.update_ticks()
    if i in [2, 5]:
        cbar.ax.set_ylabel('power density [W/m$^{2}$]')
        cbar.ax.yaxis.labelpad = 20

    # set axes to be equal in size
    ax.set_box_aspect([1, 1, 1])
    ax = set_axes_equal(ax)
    fig.tight_layout()
    
    # additional figure configuration and setting labels
    ax.set(xlabel='',
           ylabel='',
           zlabel='',
           xticks=[],
           yticks=[-0.02, 0, 0.02],
           zticks=[-0.02, 0, 0.02],
           yticklabels=[-2, 0, 2],
           zticklabels=[-2, 0, 2])
    ax.xaxis.pane.fill = False
    ax.yaxis.pane.fill = False
    ax.zaxis.pane.fill = False
    ax.view_init(1, 0)
    if show_labels:
        ax.set_ylabel('$y$ [cm]')
        ax.yaxis.labelpad = 20
        ax.zaxis.set_rotate_label(False)
        ax.set_zlabel('$z$ [cm]', rotation=90)
        ax.zaxis.labelpad = 20
    elif i in [3, 4, 5]:
        ax.set_ylabel('$y$ [cm]')
        ax.yaxis.labelpad = 20
    elif i in [0, 3]:
        ax.zaxis.set_rotate_label(False)
        ax.set_zlabel('$z$ [cm]', rotation=90)
        ax.zaxis.labelpad = 20

    # save figure
    # if show_labels:
    #     fname = os.path.join('figures',
    #                          f'{antenna}{orientation}_d{distance}mm_labels')
    # else:
    #     fname = os.path.join('figures',
    #                          f'{antenna}{orientation}_d{distance}mm')
    # save_fig(fig, fname=fname, formats=['png'], dpi=250)

# APD values

In [ ]:
ds['Antenna'] = ds['orientation'] + ' ' + ds['antenna'].str.lower()

In [ ]:
fig_config(latex=True, scaler=1.5)
sns.set(style='ticks', font='serif', font_scale=1.75)
fig, ax = plt.subplots()
ax = sns.barplot(ds.sort_values('APD_CST'),
                 x='distance', y='APD_CST', hue='Antenna',
                 palette='mako',
                 ax=ax)
for container in ax.containers:
    ax.bar_label(container, rotation=90, label_type='center', color='w')
ax.set(xlabel='distance [mm]',
       ylabel='power density [W/m$^{2}$]',
       yticks=[0, 3, 6],
       yticklabels=[0, 3, 6])
sns.move_legend(ax, 'upper left', bbox_to_anchor=(1, 0.7))
sns.despine(fig=fig, ax=ax)

# save figure
# fname = os.path.join('figures', 'apd')
# save_fig(fig, fname=fname, formats=['png'], dpi=285)